In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc
from sklearn.metrics import log_loss

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_Train = pd.read_pickle(ENV.lightgbm_train_764_nn.value)
print(X_Train.shape)
X_Test = pd.read_pickle(ENV.lightgbm_test_764_nn.value)
print(X_Test.shape)
X = pd.concat([X_Train.drop('TARGET',axis=1),X_Test])
print(X.shape)



(307511, 764)
(48744, 763)
(356255, 763)


In [3]:
X_pre_combine = pd.read_pickle(ENV.previous_app_combine_rnnALL.value)
print('previous app shape: {}'.format(X_pre_combine.shape))


X_bu_fe = pd.read_pickle(ENV.bureau_cleaned_rnnALL.value)
print('bureau shape: {}'.format(X_bu_fe.shape))

previous app shape: (1670214, 322)
bureau shape: (1716428, 124)


In [4]:
def create_main_doc(X,categorical_col):
    #1. create document
    X_cal_values = X[categorical_col].values
    X_doc = []
    for each_line in X_cal_values:
        doc = ' '.join(each_line)
        X_doc.append(doc)
        
    #2. get max feature! max unique words
    max_features = 0
    for each in tqdm(categorical_col):
        max_features+=X[each].nunique()
    print('number of unique category is : {}'.format(max_features))
    
    #3. word to sequence
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(X_doc)
    maxlen = len(categorical_col)
    X_cal=tok.texts_to_sequences(X_doc)
    X_cal = sequence.pad_sequences(X_cal,maxlen=maxlen)
    
    return X_cal,max_features

def one_hot_encoding(df,col,pre_fix,drop=True):
    df = df.copy()
    df[col] = df[col].fillna('NA_NOT_FOUND')
    col_name_list = []
    print('before encoding, shape is: {}'.format(df.shape))
    for each in df[col].unique():
        name = str(each)
        col_name = pre_fix + '_'+ name.replace(' ','_')
        col_name_list.append(col_name)
        df[col_name] = 0
        df.loc[df[col]==each,col_name] = 1
    if drop:
        df = df.drop([col],axis=1)
    print('after encoding, shape is: {}'.format(df.shape))
    return df,col_name_list

def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index,max_features,embed_size,maxlen):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  1

In [5]:
categorical_col = []
for col in X.columns:
    if X[col].dtypes == object:
        categorical_col.append(col)
print('length of categorical col is: {}'.format(len(categorical_col)))

numerical_col = list(set(X.columns) - set(categorical_col))
numerical_col.remove('SK_ID_CURR')

length of categorical col is: 37


# Scale 

In [6]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
for col in tqdm(numerical_col):
    X[col] = qt.fit_transform(X[col].values.reshape(-1,1))

100%|██████████| 725/725 [00:58<00:00, 12.45it/s]


In [7]:
X_num = X[numerical_col].values

# Process categorical column

In [8]:
X_cal,num_words_main = create_main_doc(X,categorical_col)

100%|██████████| 37/37 [00:00<00:00, 51.58it/s]


number of unique category is : 463


# Create one hot columns

In [9]:
X_one_hot = X[categorical_col].copy()
for col in tqdm(categorical_col):
    X_one_hot,new_col = one_hot_encoding(X_one_hot,col,col)
X_one_hot = X_one_hot.values    

  0%|          | 0/37 [00:00<?, ?it/s]

before encoding, shape is: (356255, 37)


  3%|▎         | 1/37 [00:02<01:28,  2.45s/it]

after encoding, shape is: (356255, 94)
before encoding, shape is: (356255, 94)


  5%|▌         | 2/37 [00:03<00:54,  1.57s/it]

...

after encoding, shape is: (356255, 453)
before encoding, shape is: (356255, 453)


 97%|█████████▋| 36/37 [01:28<00:02,  2.47s/it]

after encoding, shape is: (356255, 458)
before encoding, shape is: (356255, 458)


100%|██████████| 37/37 [01:31<00:00,  2.46s/it]

after encoding, shape is: (356255, 463)


In [10]:
X_one_hot = qt.fit_transform(X_one_hot)

In [11]:
x_num_train = X_num[:307511]
x_num_test = X_num[307511:]
x_cal_train = X_cal[:307511]
x_cal_test = X_cal[307511:]
x_onehot_train = X_one_hot[:307511]
x_onehot_test = X_one_hot[307511:]
y_train = X_Train.TARGET.values

In [12]:
dim_main_numeric = x_num_train.shape[1]
dim_main_cal = x_cal_train.shape[1]
dim_main_onehot = x_onehot_train.shape[1]
main_ebd_size = 300

# Process Previous Application

In [13]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col_pre  = ['AMT_ANNUITY',
             'AMT_APPLICATION',
             'AMT_CREDIT',
             'AMT_DOWN_PAYMENT',
             'AMT_GOODS_PRICE',
             'RATE_DOWN_PAYMENT',
             'RATE_INTEREST_PRIMARY',
             'RATE_INTEREST_PRIVILEGED',
             'SELLERPLACE_AREA',
             'CNT_PAYMENT',
             'DAYS_FIRST_DRAWING',
             'DAYS_FIRST_DUE',
             'DAYS_LAST_DUE_1ST_VERSION',
             'DAYS_LAST_DUE',
             'DAYS_TERMINATION']

trans_col_pos = ['MONTHS_BALANCE_MAX',
             'MONTHS_BALANCE_MIN',
             'MONTHS_BALANCE_SPAN',
             'CNT_INSTALMENT_MAX',
             'CNT_INSTALMENT_MIN',
             'CNT_INSTALMENT_SPAN',
             'SK_DPD_MAX',
             'SK_DPD_MIN',
             'SK_DPD_MEAN',
             'SK_DPD_COUNT',
             'SK_DPD_SUM',
             'SK_DPD_DEF_MAX',
             'SK_DPD_DEF_MIN',
             'SK_DPD_DEF_MEAN',
             'SK_DPD_DEF_COUNT',
             'SK_DPD_DEF_SUM']

trans_col_ins = ['count_VERSION',
             'count_INSTALLMENT',
             'DAY_INS_SPAN',
             'DAY_ENTRY_SPAN',
             'CNT_LATE_PAYMENT',
             'CNT_LESS_PAYMENT',
             'TOTAL_AMT_INSTALMENT',
             'TOTAL_AMT_PAYMENT',
             'INSTAL_START_DAY',
             'OWE_PORTION']

trans_col_cc = [ 'CNT_DRAWINGS_CURRENT_MAX',
       'CNT_DRAWINGS_CURRENT_MIN', 'CNT_DRAWINGS_CURRENT_MEAN',
       'AMT_TOTAL_RECEIVABLE_MAX', 'AMT_TOTAL_RECEIVABLE_MIN',
       'AMT_TOTAL_RECEIVABLE_MEAN', 'AMT_RECIVABLE_MAX', 'AMT_RECIVABLE_MIN',
       'AMT_RECIVABLE_MEAN', 'AMT_RECEIVABLE_PRINCIPAL_MAX',
       'AMT_RECEIVABLE_PRINCIPAL_MIN', 'AMT_RECEIVABLE_PRINCIPAL_MEAN',
       'CC_SK_DPD_MAX', 'CC_SK_DPD_MIN', 'CC_SK_DPD_MEAN',
       'AMT_PAYMENT_TOTAL_CURRENT_MAX', 'AMT_PAYMENT_TOTAL_CURRENT_MIN',
       'AMT_PAYMENT_TOTAL_CURRENT_MEAN', 'AMT_DRAWINGS_CURRENT_MAX',
       'AMT_DRAWINGS_CURRENT_MIN', 'AMT_DRAWINGS_CURRENT_MEAN',
       'AMT_CREDIT_LIMIT_ACTUAL_MAX', 'AMT_CREDIT_LIMIT_ACTUAL_MIN',
       'AMT_CREDIT_LIMIT_ACTUAL_MEAN', 'AMT_BALANCE_MAX', 'AMT_BALANCE_MIN',
       'AMT_BALANCE_MEAN', 'CC_MONTHS_BALANCE_MAX', 'CC_MONTHS_BALANCE_MIN',
       'MONTHS_BALANCE_MEAN', 'CC_SK_DPD_DEF_MAX', 'CC_SK_DPD_DEF_MIN',
       'CC_SK_DPD_DEF_MEAN', 'AMT_INST_MIN_REGULARITY_MAX',
       'AMT_INST_MIN_REGULARITY_MIN', 'AMT_INST_MIN_REGULARITY_MEAN',
       'CNT_INSTALMENT_MATURE_CUM_MAX', 'CNT_INSTALMENT_MATURE_CUM_MIN',
       'CNT_INSTALMENT_MATURE_CUM_MEAN', 'AMT_DRAWINGS_POS_CURRENT_MAX',
       'AMT_DRAWINGS_POS_CURRENT_MIN', 'AMT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_ATM_CURRENT_MAX', 'AMT_DRAWINGS_ATM_CURRENT_MIN',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN', 'CNT_DRAWINGS_ATM_CURRENT_MAX',
       'CNT_DRAWINGS_ATM_CURRENT_MIN', 'CNT_DRAWINGS_ATM_CURRENT_MEAN',
       'CNT_DRAWINGS_OTHER_CURRENT_MAX', 'CNT_DRAWINGS_OTHER_CURRENT_MIN',
       'CNT_DRAWINGS_OTHER_CURRENT_MEAN', 'CNT_DRAWINGS_POS_CURRENT_MAX',
       'CNT_DRAWINGS_POS_CURRENT_MIN', 'CNT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_OTHER_CURRENT_MAX', 'AMT_DRAWINGS_OTHER_CURRENT_MIN',
       'AMT_DRAWINGS_OTHER_CURRENT_MEAN', 'AMT_PAYMENT_CURRENT_MAX',
       'AMT_PAYMENT_CURRENT_MIN', 'AMT_PAYMENT_CURRENT_MEAN', 'Records_CNT',
       'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Completed',
       'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Signed',
       'NAME_CONTRACT_STATUS_Sent_proposal', 'NAME_CONTRACT_STATUS_Refused',
       'NAME_CONTRACT_STATUS_Approved']
for col in trans_col_pre + trans_col_pos +trans_col_ins +trans_col_cc:
    print(col)
    X_pre_combine[col] = qt.fit_transform(X_pre_combine[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE
RATE_DOWN_PAYMENT
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
SELLERPLACE_AREA
CNT_PAYMENT
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
MONTHS_BALANCE_MAX
MONTHS_BALANCE_MIN
MONTHS_BALANCE_SPAN
CNT_INSTALMENT_MAX
CNT_INSTALMENT_MIN
CNT_INSTALMENT_SPAN
SK_DPD_MAX
SK_DPD_MIN
SK_DPD_MEAN
SK_DPD_COUNT
SK_DPD_SUM
SK_DPD_DEF_MAX
SK_DPD_DEF_MIN
SK_DPD_DEF_MEAN
SK_DPD_DEF_COUNT
SK_DPD_DEF_SUM
count_VERSION
count_INSTALLMENT
DAY_INS_SPAN
DAY_ENTRY_SPAN
CNT_LATE_PAYMENT
CNT_LESS_PAYMENT
TOTAL_AMT_INSTALMENT
TOTAL_AMT_PAYMENT
INSTAL_START_DAY
OWE_PORTION
CNT_DRAWINGS_CURRENT_MAX
CNT_DRAWINGS_CURRENT_MIN
CNT_DRAWINGS_CURRENT_MEAN
AMT_TOTAL_RECEIVABLE_MAX
AMT_TOTAL_RECEIVABLE_MIN
AMT_TOTAL_RECEIVABLE_MEAN
AMT_RECIVABLE_MAX
AMT_RECIVABLE_MIN
AMT_RECIVABLE_MEAN
AMT_RECEIVABLE_PRINCIPAL_MAX
AMT_RECEIVABLE_PRINCIPAL_MIN
AMT_RECEIVABLE_PRINCIPAL_MEAN
CC_SK_DPD_MAX
CC_SK_DPD_MIN
CC_SK_

In [14]:
print('start')
#previous application
#10001358
max_features = 1680000
# max 73
maxlen_preapp = 12

sorted_df = X_pre_combine.sort_values(['SK_ID_CURR','DAYS_DECISION'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size_preapp = len(feature)
print('ebd size is {}'.format(embed_size_preapp))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train_preapp,x_test_preapp,y_train_preapp,embedding_matrix_preapp,num_words_preapp = get_train_ebdMat(train,test,embeddings_index,max_features,embed_size_preapp,maxlen_preapp)


start
start normalize
get embedding
ebd size is 320
create document
get embedding Mat
...get ebd mat
num of words: 1670216
(1670216, 320)


# Processs Bureau

In [15]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = [
             'AMT_ANNUITY',
             'AMT_CREDIT_MAX_OVERDUE',
             'DAYS_ENDDATE_FACT',
             'AMT_CREDIT_SUM_LIMIT',
             'AMT_CREDIT_SUM_DEBT',
             'DAYS_CREDIT_ENDDATE',
             'AMT_CREDIT_SUM',
             'CREDIT_DAY_OVERDUE',
             'CNT_CREDIT_PROLONG',
             'BUREAU_LENGTH',
             'AMT_CREDIT_SUM_OVERDUE',
             'DAYS_CREDIT_UPDATE',
    'AMT_ANNUITY_squre',
 'AMT_CREDIT_MAX_OVERDUE_squre',
 'DAYS_ENDDATE_FACT_squre',
 'AMT_CREDIT_SUM_LIMIT_squre',
 'AMT_CREDIT_SUM_DEBT_squre',
 'DAYS_CREDIT_ENDDATE_squre',
 'AMT_CREDIT_SUM_squre',
 'CREDIT_DAY_OVERDUE_squre',
 'CNT_CREDIT_PROLONG_squre',
 'BUREAU_LENGTH_squre',
 'AMT_CREDIT_SUM_OVERDUE_squre',
 'DAYS_CREDIT_UPDATE_squre',
    'AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE',
 'AMT_ANNUITY_DAYS_ENDDATE_FACT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_DEBT',
 'AMT_ANNUITY_DAYS_CREDIT_ENDDATE',
 'AMT_ANNUITY_AMT_CREDIT_SUM',
 'AMT_ANNUITY_CREDIT_DAY_OVERDUE',
 'AMT_ANNUITY_CNT_CREDIT_PROLONG',
 'AMT_ANNUITY_BUREAU_LENGTH',
 'AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE',
 'AMT_ANNUITY_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM',
 'AMT_CREDIT_MAX_OVERDUE_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_LENGTH',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_UPDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_LIMIT',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_DEBT',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM',
 'DAYS_ENDDATE_FACT_CREDIT_DAY_OVERDUE',
 'DAYS_ENDDATE_FACT_CNT_CREDIT_PROLONG',
 'DAYS_ENDDATE_FACT_BUREAU_LENGTH',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_LIMIT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_LIMIT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_DEBT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM',
 'DAYS_CREDIT_ENDDATE_CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE_CNT_CREDIT_PROLONG',
 'DAYS_CREDIT_ENDDATE_BUREAU_LENGTH',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_ENDDATE_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DAYS_CREDIT_UPDATE',
 'CREDIT_DAY_OVERDUE_CNT_CREDIT_PROLONG',
 'CREDIT_DAY_OVERDUE_BUREAU_LENGTH',
 'CREDIT_DAY_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_DAY_OVERDUE_DAYS_CREDIT_UPDATE',
 'CNT_CREDIT_PROLONG_BUREAU_LENGTH',
 'CNT_CREDIT_PROLONG_AMT_CREDIT_SUM_OVERDUE',
 'CNT_CREDIT_PROLONG_DAYS_CREDIT_UPDATE',
 'BUREAU_LENGTH_AMT_CREDIT_SUM_OVERDUE',
 'BUREAU_LENGTH_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_OVERDUE_DAYS_CREDIT_UPDATE']
for col in trans_col:
    print(col)
    X_bu_fe[col] = qt.fit_transform(X_bu_fe[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_CREDIT_MAX_OVERDUE
DAYS_ENDDATE_FACT
AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_SUM_DEBT
DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM
CREDIT_DAY_OVERDUE
CNT_CREDIT_PROLONG
BUREAU_LENGTH
AMT_CREDIT_SUM_OVERDUE
DAYS_CREDIT_UPDATE
AMT_ANNUITY_squre
AMT_CREDIT_MAX_OVERDUE_squre
DAYS_ENDDATE_FACT_squre
AMT_CREDIT_SUM_LIMIT_squre
AMT_CREDIT_SUM_DEBT_squre
DAYS_CREDIT_ENDDATE_squre
AMT_CREDIT_SUM_squre
CREDIT_DAY_OVERDUE_squre
CNT_CREDIT_PROLONG_squre
BUREAU_LENGTH_squre
AMT_CREDIT_SUM_OVERDUE_squre
DAYS_CREDIT_UPDATE_squre
AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE
AMT_ANNUITY_DAYS_ENDDATE_FACT
AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT
AMT_ANNUITY_AMT_CREDIT_SUM_DEBT
AMT_ANNUITY_DAYS_CREDIT_ENDDATE
AMT_ANNUITY_AMT_CREDIT_SUM
AMT_ANNUITY_CREDIT_DAY_OVERDUE
AMT_ANNUITY_CNT_CREDIT_PROLONG
AMT_ANNUITY_BUREAU_LENGTH
AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE
AMT_ANNUITY_DAYS_CREDIT_UPDATE
AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT
AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT
AMT_CREDIT_

In [16]:
print('start')
#previous application
#10001358
max_features = 1730000
# max 73
maxlen_bu = 13

sorted_df = X_bu_fe.sort_values(['SK_ID_CURR','DAYS_CREDIT'])
col = 'DAYS_CREDIT'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_BUREAU')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size_bu = len(feature)
print('ebd size is {}'.format(embed_size_bu))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train_bu,x_test_bu,y_train_bu,embedding_matrix_bu,num_words_bu = get_train_ebdMat(train,test,embeddings_index,max_features,embed_size_bu,maxlen_bu)


start
start normalize
get embedding
ebd size is 122
create document
get embedding Mat
...get ebd mat
num of words: 1716430
(1716430, 122)


# Create NN

In [17]:
# model = get_NN_model()
# model.summary()

In [18]:
def get_NN_model():
    main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
    main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
    main_onehot_input = Input(shape=(dim_main_onehot, ),name = 'main_onehot')
    previousapp_input = Input(shape=(maxlen_preapp, ),name = 'preapp')
    bu_input = Input(shape=(maxlen_bu, ),name = 'bu')
    
    x_preapp = Embedding(num_words_preapp, embed_size_preapp, weights=[embedding_matrix_preapp],trainable = False)(previousapp_input)
    x_preapp = BatchNormalization()(x_preapp)
    x_preapp = SpatialDropout1D(0.2)(x_preapp)
    x_preapp = Bidirectional(CuDNNGRU(8, return_sequences=True))(x_preapp)
    x_preapp = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x_preapp)
    x_preapp = BatchNormalization()(x_preapp)
    preapp_avg_pool = GlobalAveragePooling1D()(x_preapp)
    preapp_max_pool = GlobalMaxPooling1D()(x_preapp)
    x_preapp = concatenate([preapp_avg_pool, preapp_max_pool]) 
    
    
    x_bu = Embedding(num_words_bu, embed_size_bu, weights=[embedding_matrix_bu],trainable = False)(bu_input)
    x_bu = BatchNormalization()(x_bu)
    x_bu = SpatialDropout1D(0.2)(x_bu)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x_bu = Bidirectional(CuDNNGRU(8, return_sequences=True))(x_bu)
    x_bu = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x_bu)
    x_bu = BatchNormalization()(x_bu)
    bu_avg_pool = GlobalAveragePooling1D()(x_bu)
    bu_max_pool = GlobalMaxPooling1D()(x_bu)
    x_bu = concatenate([bu_avg_pool, bu_max_pool]) 
    
    
    x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
    x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
    x_main_cat = Flatten()(x_main_cat)
    
    x_main_nu = BatchNormalization()(main_numeric_input)
    x_main_nu =concatenate([x_main_nu,x_preapp,x_bu])
    x_main_nu = Dense(192,activation='relu')(x_main_nu)
    x_main_nu = Dropout(0.5)(x_main_nu)
    
    x_main_oh = Dense(512,activation='relu')(main_onehot_input)
    x_main_oh = Dropout(0.5)(x_main_oh)
    

    
    
    
    x = concatenate([x_main_nu, x_main_cat,x_main_oh])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    
    preds = Dense(1, activation="sigmoid")(x)
    model = Model([main_numeric_input,
                   main_cal_sequence,
                   main_onehot_input,
                   previousapp_input,
                   bu_input], preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
    return model

class_ratio = 1
def train_each_epoch(model,**args):
    #######################################
    x_num = args['main_numerical']
    x_cat = args['main_categorical']
    y = args['y']
    batch_size = args['batch_size']
    
    x_onehot = args['main_onehot']
    x_preapp = args['preapp']
    x_bu = args['bu']
    ########################################
    x_num,x_cat,y = shuffle(x_num,x_cat,y)
    model.fit({'main_numerical': x_num, 
               'main_categorical': x_cat,
               'main_onehot':x_onehot,
               'preapp':x_preapp,
               'bu':x_bu}, 
              y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)
    
def train_each_fold(model,**args):
    
    x_num = args['main_n_train']
    x_num_val = args['main_n_val']
    x_cat = args['main_c_train']
    x_cat_val = args['main_c_val']
    y = args['y']
    y_val = args['y_val']
    filepath = args['filepath']
    reportpath = args['reportpath']
    predspath = args['predspath']
    batch_size = args.get('batch_size',512)
    total_epoch = args.get('total_epoch',400)
    patience = args.get('patience',5)
    saving = args.get('saving',True)
    
    x_onehot = args['main_oh_train']
    x_onehot_val = args['main_oh_val']
    x_preapp = args['preapp_train']
    x_preapp_val = args['preapp_val']
    x_bu = args['bu_train']
    x_bu_val = args['bu_val']
    
    
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(model,
                                 main_numerical=x_num,
                                 main_categorical=x_cat,
                                 y=y,
                                 main_onehot = x_onehot,
                                 preapp=x_preapp,
                                 bu=x_bu,
                                 batch_size=batch_size,)
        y_pred = model.predict({'main_numerical':x_num_val,
                                'main_categorical':x_cat_val,
                                'main_onehot':x_onehot_val,
                                'preapp':x_preapp_val,
                                'bu':x_bu_val},
                               batch_size=3000,
                               verbose=1)
        score = roc_auc_score(y_val,y_pred)
        logloss = log_loss(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        print('current loss is {}'.format(logloss))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(**args):
    
    filepath = args['filepath']
    reportpath = args['reportpath']
    predspath = args['predspath']
    pred_test_file = args['predspath_test']
    batch_size = args.get('batch_size',512)
    total_epoch = args.get('total_epoch',400)
    patience = args.get('patience',5)
    saving = args.get('saving',True)
    
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(4,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra_nu = x_num_train[train_index]
        X_tra_cat = x_cal_train[train_index]
        X_val_nu = x_num_train[val_index]
        X_val_cat = x_cal_train[val_index]
        y_tra = y_train[train_index]
        y_val = y_train[val_index]
        
        x_tra_oh = x_onehot_train[train_index]
        x_val_oh = x_onehot_train[val_index]
        x_tra_preapp = x_train_preapp[train_index]
        x_val_preapp = x_train_preapp[val_index]
        x_tra_bu = x_train_bu[train_index]
        x_val_bu = x_train_bu[val_index]
        
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = filepath.format(fold)
        report_file_evl = reportpath.format(fold)
        pred_file_evl = predspath.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_NN_model()
        train_each_fold(model,
                main_n_train=X_tra_nu,
                main_n_val=X_val_nu,
                main_c_train=X_tra_cat,
                main_c_val=X_val_cat,
                y=y_tra,
                y_val=y_val,
                filepath=model_file_evl,
                reportpath=report_file_evl,
                predspath=pred_file_evl,
                batch_size=batch_size,
                total_epoch=total_epoch,
                patience=patience,
                saving=saving,
                main_oh_train=x_tra_oh,
                main_oh_val=x_val_oh,
                preapp_train=x_tra_preapp,
                preapp_val=x_val_preapp,
                bu_train=x_tra_bu,
                bu_val=x_val_bu)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict({'main_numerical':x_num_test,
                                    'main_categorical':x_cal_test,
                                    'main_onehot':x_onehot_test,
                                    'preapp':x_test_preapp,
                                    'bu':x_test_bu},
                                   batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


In [19]:
model = get_NN_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
preapp (InputLayer)             (None, 12)           0                                            
__________________________________________________________________________________________________
bu (InputLayer)                 (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 12, 320)      534469120   preapp[0][0]                     
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 13, 122)      209404460   bu[0][0]                         
__________________________________________________________________________________________________
batch_norm

In [20]:
model_file = ENV.main_ALL_nn.value
report_file = ENV.main_ALL_nn_report.value
pred_file = ENV.main_ALL_nn_preds.value
pred_test_file = ENV.main_ALL_nn_preds_test.value

train_5_folds(filepath=model_file,
              reportpath=report_file,
              predspath=pred_file,
              predspath_test=pred_test_file,
              batch_size=512,
              total_epoch=100,
              patience=20,
              saving=True)

!!!!!!!! Begin fold: 4
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/main_ALL/fold_{}.hdf5
Epoch 1/1
61502/61502 [==============================] - 1s 14us/step
saving preds...
saving report to... ../LIB/../../data/rnn/main_ALL/report_fold_4.pkl
======= current 0 / 100
previous best roc is 0
current roc is 0.7774079420397729
current loss is 0.23825845952817382
Epoch 1/1
61502/61502 [==============================] - 1s 10us/step
saving model to... ../LIB/../../data/rnn/main_ALL/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/main_ALL/report_fold_4.pkl
======= current 1 / 100
previous best roc is 0.7774079420397729
current roc is 0.7840268292708329
current loss is 0.23652194713965405
Epoch 1/1
61502/61502 [==============================] - 1s 10us/step
saving model to... ../LIB/../../data/rnn/main_ALL/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/main_ALL/report_fold_4.pkl
======= current 2 / 100
previous best roc is 0.7

In [21]:
aaaaaaa

NameError: name 'aaaaaaa' is not defined

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Flatten()(x_main_cat)
#     x_main_nu = Dense(192,activation='relu')(main_numeric_input)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Bidirectional(CuDNNGRU(8, return_sequences=True))(x_main_cat)
# #     x_main_cat = Flatten()(x_main_cat)
#     x_main_cat = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x_main_cat)
#     x_main_cat = BatchNormalization()(x_main_cat)
#     avg_pool = GlobalAveragePooling1D()(x_main_cat)
#     max_pool = GlobalMaxPooling1D()(x_main_cat)
#     x_main_cat = concatenate([avg_pool, max_pool]) 

#     x_main_nu = Dense(192,activation='relu')(main_numeric_input)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Flatten()(x_main_cat)
#     x_main_nu = BatchNormalization()(main_numeric_input)
#     x_main_nu = Dense(192,activation='relu')(x_main_nu)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

# class_ratio = 1
# def train_each_epoch(model,**args):
#     #######################################
#     x_num = args['main_numerical']
#     x_cat = args['main_categorical']
#     y = args['y']
#     batch_size = args['batch_size']
#     ########################################
#     x_num,x_cat,y = shuffle(x_num,x_cat,y)
#     model.fit({'main_numerical': x_num, 'main_categorical': x_cat}, 
#               y, 
#               batch_size=batch_size, 
#               epochs=1,
#               verbose=1,
#               class_weight={0:1,1:class_ratio})
#     return model

# def load_model(model,filepath):
#     model.load_weights(filepath)
#     return model

# def save_model(model, filepath):
#     model.save_weights(filepath)
    
# def train_each_fold(model,**args):
    
#     x_num = args['main_n_train']
#     x_num_val = args['main_n_val']
#     x_cat = args['main_c_train']
#     x_cat_val = args['main_c_val']
#     y = args['y']
#     y_val = args['y_val']
#     filepath = args['filepath']
#     reportpath = args['reportpath']
#     predspath = args['predspath']
#     batch_size = args.get('batch_size',512)
#     total_epoch = args.get('total_epoch',400)
#     patience = args.get('patience',5)
#     saving = args.get('saving',True)
    
    
#     ROC_AUC_SCORE = []
#     for epoch in range(total_epoch):  
#         model = train_each_epoch(model,
#                                  main_numerical=x_num,
#                                  main_categorical=x_cat,
#                                  y=y,
#                                  batch_size=batch_size)
#         y_pred = model.predict({'main_numerical':x_num_val,
#                                 'main_categorical':x_cat_val},
#                                batch_size=3000,
#                                verbose=1)
#         score = roc_auc_score(y_val,y_pred)
#         logloss = log_loss(y_val,y_pred)
#         if len(ROC_AUC_SCORE) == 0:
#             if saving:
#                 save_model(model,filepath)
#             best_score = 0 
#             if saving:
#                 print('saving preds...')
#                 pickle.dump(y_pred,open(predspath,'wb'))
#         else:
#             best_score = max(ROC_AUC_SCORE)
#             if score >= best_score:
#                 if saving:
#                     print('saving model to... {}'.format(filepath))
#                     save_model(model,filepath)
#                     print('saving preds...')
#                     pickle.dump(y_pred,open(predspath,'wb'))
#         ROC_AUC_SCORE.append(score)
#         if saving:
#             print('saving report to... {}'.format(reportpath))
#             pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
#         print('======= current {} / {}'.format(epoch,total_epoch))
#         print('previous best roc is {}'.format(best_score))
#         print('current roc is {}'.format(score))
#         print('current loss is {}'.format(logloss))
#         try:
#             best_round = ROC_AUC_SCORE.index(best_score)
#         except ValueError:
#             best_round = -1
#         if len(ROC_AUC_SCORE) > patience + best_round:
#             print('reach patience! end loop')
#             break
            
# def train_5_folds(**args):
    
#     filepath = args['filepath']
#     reportpath = args['reportpath']
#     predspath = args['predspath']
#     pred_test_file = args['predspath_test']
#     batch_size = args.get('batch_size',512)
#     total_epoch = args.get('total_epoch',400)
#     patience = args.get('patience',5)
#     saving = args.get('saving',True)
    
#     train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
#     val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

#     for fold in range(0,len(train_fold_index)):
#         print('!!!!!!!! Begin fold: {}'.format(fold))
#         train_index = train_fold_index[fold]
#         val_index = val_fold_index[fold]
#         X_tra_nu = x_num_train[train_index]
#         X_tra_cat = x_cal_train[train_index]
#         X_val_nu = x_num_train[val_index]
#         X_val_cat = x_cal_train[val_index]
#         y_tra = y_train[train_index]
#         y_val = y_train[val_index]
#         print('preparing train/val done!')
#         print('before evaluating: {}'.format(model_file))
#         model_file_evl = filepath.format(fold)
#         report_file_evl = reportpath.format(fold)
#         pred_file_evl = predspath.format(fold)
#         pred_test_file_evl = pred_test_file.format(fold)
#         model = get_NN_model()
#         train_each_fold(model,
#                 main_n_train=X_tra_nu,
#                 main_n_val=X_val_nu,
#                 main_c_train=X_tra_cat,
#                 main_c_val=X_val_cat,
#                 y=y_tra,
#                 y_val=y_val,
#                 filepath=model_file_evl,
#                 reportpath=report_file_evl,
#                 predspath=pred_file_evl,
#                 batch_size=batch_size,
#                 total_epoch=total_epoch,
#                 patience=patience,
#                 saving=saving)
#         gc.collect()
#         #### predict test
#         model = load_model(model,model_file_evl)
#         test_preds = model.predict({'main_numerical':x_num_test,
#                                 'main_categorical':x_cal_test},batch_size=1500,verbose=1)
#         pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
#         print('\n')
